In [ ]:
import numpy as np 
import math
import os
import mne
import matplotlib
from tqdm.notebook import tqdm
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests
import matplotlib as mpl
import matplotlib.lines as mlines
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats as sp
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


cmap = sns.color_palette("colorblind")
%run -i colorblind.py

In [ ]:
cmap_new = rainbow_colors(12)
fig, ax = plt.subplots(1,1,figsize=(3,1))
for i, color in enumerate(cmap_new): 
    plt.scatter(i, 1, color=color)

In [ ]:
def corr_datscan(crit, datscan, age): 
    rho = np.zeros((30, len(datscan)))
    p = np.zeros_like(rho)

    for i_dat, dat in enumerate(datscan): 

        # Check nan in datscan file
        idx_dat = np.argwhere(np.isnan(datscan))
        dats = np.delete(dat, idx_dat)
        crit_dats = np.delete(crit, idx_dat, axis=0)
        age_dats = np.delete(age, idx_dat, axis=0)
        
        # Spearman's correlation
        for iF in range(30):
            rho[iF, i_dat], p[iF, i_dat] = sp.spearmanr(crit_dats[:,iF], dats)
        
    # Benjiamini - Hochberg's correction
    _, p_corrected, _, _ = multipletests(p.ravel(), method='fdr_bh')

    p_corr = p_corrected.reshape([30,2])
       
    return rho, p, p_corr

def corr_datscan_single_Ch(crit, datscan, age): 
    _, nF = crit.shape
    rho = np.zeros((nF, len(datscan)))
    p = np.zeros_like(rho)

    for i_dat, dat in enumerate(datscan): 

        # Check nan in datscan file
        idx_dat = np.argwhere(np.isnan(datscan))
        dats = np.delete(dat, idx_dat)
        crit_dats = np.delete(crit, idx_dat, axis=0)
        age_dats = np.delete(age, idx_dat, axis=0)
        
        # Spearman's correlation
        for iF in range(crit_dats.shape[1]):
            rho[iF, i_dat], p[iF, i_dat] = sp.spearmanr(crit_dats[:,iF], dats, nan_policy='omit')
        
            
    return rho, p


def freq_band_extraction(data1, data2, freq_range):
    data1 = data1[:,:,freq_range].mean(axis=2)
    data2 = data2[:,:,freq_range].mean(axis=2)
    data = np.concatenate([data1, data2])

    return data

def extract_fei_new(files):
    
    data, name_subjs, data_masked = list(), list(), list()
    for ifile, file in enumerate(tqdm(files)):
        
        name_subjs.append(file.split('/')[9][:6])
        
        tmp = np.load(file, allow_pickle=True)

        data.append(tmp[1]) 
        data_masked.append(np.nanmean(tmp[1,...],axis=0)) # Average channels without nan
    
    fEI_ch = np.array(data)
    fEI_masked = np.array(data_masked)

    return name_subjs, fEI_ch, fEI_masked



#### Set input

In [ ]:
# Set folder data path and output path
rbd_root = '.../RBD_root/'
path_out = '.../Results/'

lut_filename = 'lut.xlsx'

# Set frequency limits
f_min = 2
f_max = 90
scale_freq='log'


#### Extract dopamine and fEI information

In [ ]:
# Import LOOK UP TABLE - RBD 
lut_rbd = pd.read_excel(rbd_root + lut_filename).set_index('ID')

In [ ]:
# Extract age
age_r1 = lut_rbd.loc[(lut_rbd['REC']=='B')]['AGE to the acquisition'].to_numpy()
age_r2 = lut_rbd.loc[(lut_rbd['REC']=='FU1')]['AGE to the acquisition'].to_numpy()
age = np.concatenate((age_r1,age_r2))

In [ ]:
# Extract information about conveterted and not converted patients
name_lateRBD_r1 = lut_rbd.loc[(~lut_rbd['CONVERSION TO '].isnull()) & (lut_rbd['REC'] == 'B') & (lut_rbd['CONV (0=non, 1=late, 2=early)'] == 1) ].index.to_list()
name_lateRBD_r1 = [i.split('_', 1)[0] for i in name_lateRBD_r1]
name_lateRBD_r2 = lut_rbd.loc[(~lut_rbd['CONVERSION TO '].isnull()) & (lut_rbd['REC'] == 'FU1') & (lut_rbd['CONV (0=non, 1=late, 2=early)'] == 1)  ].index.to_list()
name_lateRBD_r2 = [i.split('_', 1)[0] for i in name_lateRBD_r2]

name_earlyRBD_r1 = lut_rbd.loc[(~lut_rbd['CONVERSION TO '].isnull()) & (lut_rbd['REC'] == 'B') & (lut_rbd['CONV (0=non, 1=late, 2=early)'] == 2) ].index.to_list()
name_earlyRBD_r1 = [i.split('_', 1)[0] for i in name_earlyRBD_r1]
name_earlyRBD_r2 = lut_rbd.loc[(~lut_rbd['CONVERSION TO '].isnull()) & (lut_rbd['REC'] == 'FU1') & (lut_rbd['CONV (0=non, 1=late, 2=early)'] == 2) ].index.to_list()
name_earlyRBD_r2 = [i.split('_', 1)[0] for i in name_earlyRBD_r2]

name_sRBD_r1 = lut_rbd.loc[(lut_rbd['CONVERSION TO '].isnull()) & (lut_rbd['REC'] == 'B') & (lut_rbd['CONV (0=non, 1=late, 2=early)'] == 0)].index.to_list()
name_sRBD_r1 = [i.split('_', 1)[0] for i in name_sRBD_r1]
name_sRBD_r2 = lut_rbd.loc[(lut_rbd['CONVERSION TO '].isnull()) & (lut_rbd['REC'] == 'FU1') & (lut_rbd['REC'] == 'B') & (lut_rbd['CONV (0=non, 1=late, 2=early)'] == 0)].index.to_list()
name_sRBD_r2 = [i.split('_', 1)[0] for i in name_sRBD_r2]

In [ ]:
# Extract information about dopamine levels 
caudL_r1 = lut_rbd['Caudate left'].loc[lut_rbd['REC']=='B'].to_numpy()
caudR_r1 = lut_rbd['Caudate right'].loc[lut_rbd['REC']=='B'].to_numpy()

putL_r1 = lut_rbd['Putamen left'].loc[lut_rbd['REC']=='B'].to_numpy()
putR_r1 = lut_rbd['Putamen right'].loc[lut_rbd['REC']=='B'].to_numpy()

dat_r1 = [(putL_r1 + putR_r1)/2, (caudL_r1 + caudR_r1)/2]

caudL_r2 = lut_rbd['Caudate left'].loc[lut_rbd['REC']=='FU1'].to_numpy()
caudR_r2 = lut_rbd['Caudate right'].loc[lut_rbd['REC']=='FU1'].to_numpy()

putL_r2 = lut_rbd['Putamen left'].loc[lut_rbd['REC']=='FU1'].to_numpy()
putR_r2 = lut_rbd['Putamen right'].loc[lut_rbd['REC']=='FU1'].to_numpy()

dat_r2 = [(putL_r2 + putR_r2)/2, (caudL_r2 + caudR_r2)/2]

datscan = np.hstack((dat_r1, dat_r2))

In [ ]:
# MMSE
id_r1 = lut_rbd.loc[(lut_rbd['REC'] == 'B')].index
mmse_r1 = lut_rbd.loc[id_r1, 'MMSE'].to_numpy()

id_r2 = lut_rbd.loc[(lut_rbd['REC'] == 'FU1')].index
mmse_r2 = lut_rbd.loc[id_r2, 'MMSE'].to_numpy()

mmse = np.concatenate((mmse_r1, mmse_r2))

# UPDRS-III
id_r1 = lut_rbd.loc[(lut_rbd['REC'] == 'B')].index
updrs_r1 = lut_rbd.loc[id_r1, 'UPDRS-III'].to_numpy()

id_r2 = lut_rbd.loc[(lut_rbd['REC'] == 'FU1')].index
updrs_r2 = lut_rbd.loc[id_r2, 'UPDRS-III'].to_numpy()

updrs = np.concatenate((updrs_r1, updrs_r2))

In [ ]:
# Extract fEI

name_subj_r1, fei_ch_r1, fei_r1 = extract_fei_new(sorted(glob.glob(os.path.join(rbd_root + 'fei/run-01/','*fei06*'))))
name_subj_r2, fei_ch_r2, fei_r2 = extract_fei_new(sorted(glob.glob(os.path.join(rbd_root + 'fei/run-02/','*fei06*'))))

In [ ]:
# Set frequency axis
freq_vals, _ = create_frequency_axis(f_min=f_min, f_max=f_max, scale_freq =scale_freq)

In [ ]:
freq_vals = freq_vals[:30]
fei_r1 = fei_r1[:,:30]
fei_r2 = fei_r2[:,:30]
fei = np.concatenate((fei_r1,fei_r2))

In [ ]:
# fEI in specific frequency band

fei_ch_r1_f = fei_ch_r1[..., 7:10].mean(axis=2)
fei_ch_r2_f = fei_ch_r2[..., 7:10].mean(axis=2)

fei_ch_f = np.concatenate((fei_ch_r1_f, fei_ch_r2_f))


#### Channel File

In [ ]:
# Channel positions and names
subj_file = '.../sub-01_task-rest_run-01_laplacian_eeg-raw.fif'
raw = mne.io.read_raw_fif(subj_file, preload=True, verbose=False)

info_pos = raw.info
ch_names = info_pos['ch_names'][:-2] # remove no-EEG channels

In [ ]:
fei_ch = np.concatenate((fei_ch_r1[...,:30],fei_ch_r2[...,:30]))

#### Correlation dopamine levels vs fEI

In [ ]:
rho_fei, p_fei, p_fei_corrected = corr_datscan(fei, datscan, age)

#### Correlation across frequency

In [ ]:
cc_fei_f, p_fei_f = spearmanr(fei, nan_policy='omit', alternative='two-sided')

In [ ]:
rho_fei_theta, p_fei_theta = spearmanr(fei[:,7:10].mean(axis=1), datscan[0,:], nan_policy='omit', alternative='two-sided')

#### Correlation with clinical scores

In [ ]:
#Correlazione fEI vs updrs-iii
from scipy.stats import kendalltau
rho_updrs = np.zeros((len(freq_vals),))
p_updrs = np.zeros_like(rho_updrs)

for i, iF in enumerate(freq_vals): 
    
    rho_updrs[i], p_updrs[i] = kendalltau(fei[:,i], updrs, nan_policy='omit')
    
_, p_corrected_updrs, _, _ = multipletests(p_updrs, method='fdr_bh')

#Correlazione fEI vs mmse
rho_mmse = np.zeros((len(freq_vals),))
p_mmse = np.zeros_like(rho_mmse)

for i, iF in enumerate(freq_vals): 
    
    rho_mmse[i], p_mmse[i] = kendalltau(fei[:,i], mmse, nan_policy='omit')
    
_, p_corrected_mmse, _, _ = multipletests(p_mmse, method='fdr_bh')

#### Correlation single channels - all frequencies

In [ ]:
_, nCh, nF = fei_ch.shape
rho_fei_ch = np.zeros((nCh, nF, 2)); p_fei_ch = np.zeros((nCh, nF, 2));

# Spearman Correlation
for iCh in range(nCh):
    rho_fei_ch[iCh,:,:], p_fei_ch[iCh,:,:] = corr_datscan_single_Ch(fei_ch[:,iCh,:], datscan,age)

# Benjiamini - Hochberg's correction
_,  p_fei_ch_corrected, _, _ = multipletests(p_fei_ch.ravel(), method='fdr_bh')
p_fei_ch_corrected = p_fei_ch_corrected.reshape([60,30,2])

#### Correlation single channels - band frequencies

In [ ]:
# Spearman Correlation
rho_fei_ch_f = np.zeros((nCh, 2))
p_fei_ch_f = np.zeros_like(rho_fei_ch_f)

for iCh in range(nCh):
    rho_fei_ch_f[iCh,:], p_fei_ch_f[iCh,:] = corr_datscan_single_Ch(fei_ch_f[:,iCh][np.newaxis].T, datscan, age)
    
# Benjiamini - Hochberg's correction
_,  p_fei_ch_f_corrected, _, _ = multipletests(p_fei_ch_f.ravel(), method='fdr_bh')
p_fei_ch_f_corrected = p_fei_ch_f_corrected.reshape([nCh,2])

#### Index converted - not converted

In [ ]:
idx_r1_lateRBD = [name_subj_r1.index(x) for x in name_lateRBD_r1 if x in name_subj_r1]
idx_r2_lateRBD = [name_subj_r2.index(x) for x in name_lateRBD_r2 if x in name_subj_r2]

idx_r1_earlyRBD = [name_subj_r1.index(x) for x in name_earlyRBD_r2 if x in name_subj_r1]
idx_r2_earlyRBD = [name_subj_r2.index(x) for x in name_earlyRBD_r2 if x in name_subj_r2]

idx_r1_sRBD = [name_subj_r1.index(x) for x in name_sRBD_r1 if x in name_subj_r1]
idx_r2_sRBD = [name_subj_r2.index(x) for x in name_sRBD_r2 if x in name_subj_r2]

#### Figure 2

In [ ]:
# Set input for plot
column = ['Putamen', 'Caudate']
colors = [cmap[6], cmap[9]]
i = [0.02, -0.02]

In [ ]:
# Create legend
handles = list()
handles.append(mlines.Line2D([], [], color=cmap[6], label='Putamen'))
handles.append(mlines.Line2D([], [], color=cmap[9], label='Caudate'))

handles.append(mlines.Line2D([], [], color='black', marker='o', linestyle='none',
                        markersize=10, label='pvalue < 0.05',markerfacecolor='none'))

handles.append(mlines.Line2D([], [], color='black', marker='o', linestyle='none',
                        markersize=10, label='pvalue corrected < 0.05'))

In [ ]:
import seaborn as sns
import pandas as pd


fig = plt.figure(figsize=(17.6/2.54,8/2.54), layout='constrained')

# Panel description
panel = fig.subfigures(2, 1)

# Figure top
figs_top = panel[0].subfigures(1, 2)

ax1 = figs_top[0].subplots(1,1)

m_size=6
l_size=8
a_size=6

# Correlation fEI vs dop.levels
for i_col, name_col in enumerate(column): 
    ax1.semilogx(freq_vals, rho_fei[:,i_col], label=column[i_col], color=colors[i_col]);
    for iF, nF in enumerate(freq_vals):
        if p_fei[iF, i_col] < 0.05: 
            ax1.scatter(nF, 0.25+i[i_col], marker='o', s=m_size,  
                        facecolors='none', edgecolors=colors[i_col])
        if p_fei_corrected[iF, i_col] < 0.05: 
            ax1.scatter(nF, 0.25+i[i_col], facecolors=colors[i_col],
                        edgecolors=colors[i_col], marker='o', s=m_size)

ax1.tick_params(labelsize=a_size)
ax1.set_ylim([-.5, .3])
#ax1.set_xticks([2,5,10,20,50,70], [2,5,10,20,50,70])
ax1.set_ylabel('CC - fEI vs SBR', fontsize=l_size)
ax1.set_xlabel('Frequencies [Hz]', fontsize=l_size)
legend = ax1.legend(loc='upper left', fontsize=a_size)
legend.get_frame().set_edgecolor('none')
legend.get_frame().set_facecolor('none')



# Scatter plot
ax2 = figs_top[1].subplots(1,1)
#ax2.axvline(x=0.87, color='black', linestyle='--', linewidth=2)

df = pd.DataFrame({'fEI (5-7)Hz':fei[:,7:10].mean(axis=1),
                   'Putamen':datscan[0]})
sns.regplot(x='fEI (5-7)Hz', y='Putamen', data=df, fit_reg=True, ci=95, n_boot=1000, 
           scatter_kws={'color':'white', 's':m_size}, line_kws={'color': cmap[7]}, ax=ax2)


# sRBD
ax2.scatter(fei_r1[idx_r1_sRBD,8:11].mean(axis=1), 
            dat_r1[0][idx_r1_sRBD], marker='o', color=cmap_new[5], 
            label='non-converters', s=m_size)
ax2.scatter(fei_r2[idx_r2_sRBD,8:11].mean(axis=1), 
            dat_r2[0][idx_r2_sRBD], marker='o', color=cmap_new[5], s=m_size)

# late RBD 
ax2.scatter(fei_r1[idx_r1_lateRBD,8:11].mean(axis=1), 
            dat_r1[0][idx_r1_lateRBD], marker='^', 
            color=cmap_new[9], label='late-converters', s=m_size) 
ax2.scatter(fei_r2[idx_r2_lateRBD,8:11].mean(axis=1), 
            dat_r2[0][idx_r2_lateRBD], marker='^', 
            color=cmap_new[9], s=m_size) 

# early RBD
ax2.scatter(fei_r1[idx_r1_earlyRBD,8:11].mean(axis=1), 
            dat_r1[0][idx_r1_earlyRBD], marker='X', color=cmap_new[11], 
            label='early-converters', s=m_size)
ax2.scatter(fei_r2[idx_r2_earlyRBD,8:11].mean(axis=1), 
            dat_r2[0][idx_r2_earlyRBD], marker='X', color=cmap_new[11], 
            s=m_size) 

ax2.tick_params(labelsize=a_size)
ax2.set_ylabel('SBR - Putamen', fontsize=l_size)
ax2.set_xlabel('fEI (5-7Hz)', fontsize=l_size)

legend = ax2.legend(loc='lower left', fontsize=a_size);
legend.get_frame().set_facecolor('none')
legend.get_frame().set_edgecolor('none')


figs_bottom = panel[1].subfigures(1, 2)

ax3 = figs_bottom[0].subplots(1,1)

# Correlation fEI vs clinical score
ax3.semilogx(freq_vals, rho_mmse, label='MMSE', color=cmap[5]);
ax3.semilogx(freq_vals, rho_updrs, label='MDS-UPDRS-III', color=cmap[7]);

for iF, nF in enumerate(freq_vals):
    if p_mmse[iF] < 0.05: 
        ax3.scatter(nF, 0.25, marker='o', 
                    facecolors='none', edgecolors=cmap[5], s=m_size)
    if p_corrected_mmse[iF] < 0.05: 
        ax3.scatter(nF, 0.25, facecolors=cmap[5],
                    edgecolors=colors[0], marker='o', s=m_size)
        
for iF, nF in enumerate(freq_vals):
    if p_updrs[iF] < 0.05: 
        ax3.scatter(nF, 0.27, marker='o', 
                    facecolors='none', edgecolors=cmap[7], s=m_size)
    if p_corrected_updrs[iF] < 0.05: 
        ax3.scatter(nF, 0.27, facecolors=cmap[7],
                    edgecolors=cmap[7], marker='o', s=m_size)

ax3.tick_params(labelsize=a_size)
ax3.set_ylim([-.2, .3])
#ax3.set_xticks([2,5,10,20,50,70], [2,5,10,20,50,70])
ax3.set_ylabel('CC - fEI vs clinical scores', fontsize=l_size)
ax3.set_xlabel('Frequencies [Hz]', fontsize=l_size)
legend = ax3.legend(loc='upper left', fontsize=a_size)
legend.get_frame().set_edgecolor('none')
legend.get_frame().set_facecolor('none')

ax4 = figs_bottom[1].subplots(1,2, gridspec_kw={'width_ratios': [.9, .1]})


# Topographies
_, j_range = p_fei_ch_f_corrected.shape

#for j in range(j_range):
mask = p_fei_ch_f_corrected[:,0] < 0.05
im, _ = mne.viz.plot_topomap(rho_fei_ch_f[:,0], info_pos, 
                                  axes=ax4[0], show=False, 
                                  cmap=plt.get_cmap('colorblind_diverging'), vlim=(-.4,.4),
                                  ch_type='eeg', mask=mask, 
                                  mask_params=dict(markersize=m_size, markeredgecolor='k',markerfacecolor='w'));
    


norm = mpl.colors.Normalize(vmin=-.4, vmax=.4)
cb1 = mpl.colorbar.ColorbarBase(ax=ax4[1], cmap=plt.get_cmap('colorblind_diverging'),norm=norm,
                                orientation='vertical')

cb1.ax.tick_params(labelsize=a_size); 
cb1.ax.set_yticks([-.4, -.2,0,.2, .4]); 
cb1.set_label('CC', fontsize=a_size)

ax1.set_xticks([2,5,10,20,50,70], [2,5,10,20,50,70], fontsize=a_size)
ax3.set_xticks([2,5,10,20,50,70], [2,5,10,20,50,70], fontsize=a_size)


ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)

#plt.subplots_adjust(left=.05, bottom=.15, right=None, top=None, wspace=None, hspace=None)

plt.savefig(path_out + 'Crit_F3.svg', dpi=600, bbox_inches = "tight")
plt.savefig(path_out + 'Crit_F3.png', dpi=300, bbox_inches = "tight")

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()
df['Frequencies [Hz]'] = freq_vals
df['$\rho$ (DaT putamen vs fEI)'] = rho_fei[:,0]
df['$\rho$ (DaT caudate vs fEI)'] = rho_fei[:,1]
df['$\it{p}$ (DaT putamen vs fEI)'] = p_fei[:,0]
df['$\it{p}$ (DaT caudate vs fEI)'] = p_fei[:,1]
df['$\it{p}$ corrected (DaT putamen vs fEI)'] = p_fei_corrected[:,0]
df['$\it{p}$ corrected (DaT caudate vs fEI)'] = p_fei_corrected[:,1]

df1 = pd.DataFrame()
df1['Frequencies [Hz]'] = freq_vals
df1['$\rho$ (MMSE vs fEI)'] = rho_mmse
df1['$\it{p}$ (MMSE vs fEI)'] = p_mmse
df1['$\it{p}$ corrected (MMSE vs fEI)'] = p_corrected_mmse
df1['$\rho$ (MDS-UPDRS-III vs fEI)'] = rho_updrs
df1['$\it{p}$ (MDS-UPDRS-III vs fEI)'] = p_updrs
df1['$\it{p}$ corrected (MDS-UPDRS-III vs fEI)'] = p_corrected_updrs

df_topo = pd.DataFrame()
df_topo['$\rho$ (DaT putamen vs fEI [5-7 Hz])'] = rho_fei_ch_f[:,0]
df_topo['$\it{p}$ (DaT putamen vs fEI [5-7 Hz])'] = p_fei_ch_f[:,0]
df_topo['$\it{p}$ corrected (DaT putamen vs fEI [5-7 Hz])'] = p_fei_ch_f_corrected[:,0]

with pd.ExcelWriter('SupplementaryTables_F4.xlsx') as writer:
    df.set_index('Frequencies [Hz]').to_excel(writer, sheet_name='Figure4A')
    df1.set_index('Frequencies [Hz]').to_excel(writer, sheet_name='Figure4C')
    df_topo.to_excel(writer, sheet_name='Figure4D')